https://github.com/monologg/KoELECTRA

In [1]:
# import library
import pandas as pd
from tqdm import tqdm
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline

In [2]:
ice = pd.read_csv('data/icecream_emot.csv')
tsh = pd.read_csv('data/sherpa_emot3.csv')

In [3]:
tsh = tsh[['title_comments', 'star']]
tsh.rename(columns={'title_comments' : 'text', 'star' : 'score'}, inplace=True)
tsh['score'] = tsh['score'].apply(lambda x : int(x.replace('점','')))
tsh['score'] = tsh['score'].apply(lambda x : 0 if x == 3 else (1 if x > 3 else -1))
tsh

,text,score
0,법정연수 유용하네요 의무적으로 해마다 듣는 연수인데 다른 강사에게 듣고 싶어 신청했...,1
1,"독립운동에 관한 지식 쌓기 독립운동에 대해서 알고, 학생 대상으로 수업하기 좋았습니...",1
2,삶을 바꾸는 사회수업 주제통합 질문과 상상으로 진행하기 사회 교과서 개정되고 학습량...,1
3,영어 회화가 막막한 선생님들께 유익한 연수! 영어가 너무 오랜만이시거나 일상생활에서...,1
4,독서교육을 해보고 싶어지게 만드는 연수!!! 아이를 키우며 그림책과 동화책에 관심을...,1
...,...,...
2595,알찬연수 고맙습니다.^^ 학교 진로교육의 목표 (2015개정)학생 자신의 진로를 창...,1
2596,"스토리텔링을 활용한 초등수학 수업 연수 굿굿! 선생님들의 수학 개념 설명을 듣고, ...",1
2597,고맙습니다. 진로라는게 흥미는 있지만 너무 포괄적이네요.,1
2598,감사합니다^^ 진로교육에 관해 많은 것을 알게 되었습니다.\n감사합니다.,1


In [4]:
ice = ice[['title_comments', 'rated']]
ice.rename(columns={'title_comments' : 'text', 'rated' : 'score'}, inplace=True)
ice['score'] = ice['score'].apply(lambda x : int(x[-1]))
ice['score'] = ice['score'].apply(lambda x : 0 if x == 3 else (1 if x > 3 else -1))
ice

,text,score
0,알차네요 무엇보다도 빨리들을 수 있어 좋습니다,1
1,명불허전 명강사라 그런지 말씀을 참 잘하시네요~,1
2,강추 30년 가까이 일하면서도 놓치고 있었던 것들을 알려주었습니다. 그리고 무조건 ...,1
3,패들렛과 띵커벨 활용 잘 할 수 있어요 패들렛과 띵커벨 활용하고 있었으나 다양한 교...,1
4,"필수 연수 부담 없이 편안하게 안전 연수 올 해 꼭 받아야 하는데, 부담 없이 편안...",1
...,...,...
5236,알베르토가 함께하는 핵심역량 연계 다문화 교육 여타 연수와 다르게 새로 알게된 내용...,1
5237,전문가가 알려주는 창의 영재교육의 비법 연수 후기 영재 교육에 대한 틀을 잡아주시는...,1
5238,도도한 교사생활 연수를 통해 자신감을 더 갖게 되었다.,1
5239,교실에서 활용할 수 있는 앱들 중심으로 재미있게 들었어요. 교실에서 활용할 수 있는...,1


tsh modeling

In [33]:

model_lst = ['jaehyeong/koelectra-base-v3-generalized-sentiment-analysis', 
            'klue/bert-base', 
            'matthewburke/korean_sentiment',
            # 'monologg/koelectra-base-discriminator',
            # "monologg/koelectra-small-discriminator",
            # "monologg/koelectra-base-v2-discriminator",
            # "monologg/koelectra-small-v2-discriminator",
            # "monologg/koelectra-base-v3-discriminator",
            # "monologg/koelectra-small-v3-discriminator",
            # "facebook/bart-large-mnli",
            # "monologg/koelectra-base-v3-goemotions",
            'skt/kogpt2-base-v2',
            # 'skt/kobert-base-v1',
            'beomi/kcbert-large',
            # "google/electra-small-discriminator"
            ]


# for i in model_lst[3]: # 4~
    # load model
pre_model = model_lst[3]
tokenizer = AutoTokenizer.from_pretrained(pre_model)
model = AutoModelForSequenceClassification.from_pretrained(pre_model)
sentiment_classifier = TextClassificationPipeline(tokenizer=tokenizer, model=model)

print('====='+ '\033[95m' + pre_model + '\033[0m' + '=====')

# target reviews
review_list = list(ice.text)

ice[pre_model] = '-'
# predict
for idx, review in tqdm(enumerate(review_list)):
    pred = sentiment_classifier(review)
    print(f'{review}\n>> {pred[0]}')
    # if (pred[0]['label']=='1')&(pred[0]['score'] > 0.51) | (pred[0]['label']=='LABEL_1')&(pred[0]['score'] > 0.51):
    #     ice[pre_model][idx] = '1'
    # elif (pred[0]['label']=='0')&(pred[0]['score']>0.51) | (pred[0]['label']=='LABEL_0')&(pred[0]['score'] > 0.51):  
    #     ice[pre_model][idx] = '-1'
    # else:
    #     ice[pre_model][idx] = '0'

    # print(f'{review}\n>> {pred[0]}')

print('🥕')
print('')


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v2-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


=====monologg/koelectra-base-v2-discriminator=====


3it [00:00, 23.55it/s]

알차네요 무엇보다도 빨리들을 수 있어 좋습니다
>> {'label': 'LABEL_0', 'score': 0.5336449146270752}
명불허전 명강사라 그런지 말씀을 참 잘하시네요~
>> {'label': 'LABEL_0', 'score': 0.5437445640563965}
강추 30년 가까이 일하면서도 놓치고 있었던 것들을 알려주었습니다. 그리고 무조건 교사 탓만 하는 학부모에게 이론적으로 내세울 수 있는 내용을 많이 얻었습니다.
그래서, 매우 만족합니다. 다른 샘들도 꼭 한 번 들어보세요.

역시 아이스크림 연수는 한 번도 저를 실망시키지 않는군요. 엄지척!
>> {'label': 'LABEL_0', 'score': 0.5433538556098938}
패들렛과 띵커벨 활용 잘 할 수 있어요 패들렛과 띵커벨 활용하고 있었으나 다양한 교과에 특히 독서 지도에 많이 활용 할 수 있게 되었어요
>> {'label': 'LABEL_0', 'score': 0.5442795157432556}
필수 연수 부담 없이 편안하게 안전 연수 올 해 꼭 받아야 하는데, 부담 없이 편안한 마음으로 마쳤어요.~
>> {'label': 'LABEL_0', 'score': 0.5501039624214172}


6it [00:00, 24.46it/s]

...

유익한 강의 감사합니다 :) 타로를 통해 상담할 수 있다는 것이 너무 신기했습니다. 내용도 너무 알차서 즐거웠습니다 :)
>> {'label': 'LABEL_0', 'score': 0.5449248552322388}
직업탐구 연수 후기 다양한 직업들에 대해 알 수 있는 연수였습니다
추후 학생들의 진로 교육에 도움이 될 것이라 기대합니다
>> {'label': 'LABEL_0', 'score': 0.5523996949195862}
세계사를 아는 데 큰 도움이 되었습니다. 세계사를 아는 데 큰 도움이 되었습니다.
>> {'label': 'LABEL_0', 'score': 0.5559288859367371}
유익한 강의였습니다. 유익한 강의였습니다. 연수 종료 후 다시 한번 처음부터 강의를 들을 예정입니다. 감사합니다.
>> {'label': 'LABEL_0', 'score': 0.5457218885421753}
유익한 연수였습니다. 아이들의 다양한 심리에 대해 알 수 있었고, 대처하는 방법도 알 수 있어 유익했습니다.
>> {'label': 'LABEL_0', 'score': 0.5455822944641113}


413it [00:19, 18.96it/s]

교사에게 필요한 힐링의 시간 모든 동작을 잘 할 수는 없었지만 가능한 한 따라하려고 노력했고, 자주 하다보니 조금씩 유연해지는 느낌으로 변화가 시작되었음. 연수는 끝났지만 복습을 통해 꾸준히 노력하여 건강을 유지하거나 회복하는데 활용해야겠음. 좋은 연수 감사합니다.
>> {'label': 'LABEL_0', 'score': 0.5506322979927063}
실무와 관련 있는 연수! 실무에 유용한 내용이 많아 좋습니다.
>> {'label': 'LABEL_0', 'score': 0.5499070882797241}
교사가 격려받는 힐링수업 들으면서 제가 힐링이 되는 시간이었습니다. 1년간 다시 보기가 가능하니 힘들때마다 들어와서 격려받고 새힘을 받을 수 있어 좋네요. 오랫만에 동료들과 오프라인 연수를 듣는 기분이었습니다. 강사 선생님들 한분 한분 정성을 다해 수업해주시는 것이 느껴져 특히 좋았습니다. 격려 수업!! 최고예요!!! 나이스 연수재수강이 가능한 3년에 한번씩은 들어야겠습니다.^^
>> {'label': 'LABEL_0', 'score': 0.550492525100708}
고민 톡톡! 토크로 풀어보는 재밌는 성[性] 이야기 성에 대하여 허심탄하게 애기할 수 있고 성의 주체가 본인이 되어야 하고 좋고 싫다라는 의견을 자유롭게 말할 수 있는 사회적 분위가 조성되어져야 함을 다시 한번 깨닫게 되었다.
>> {'label': 'LABEL_0', 'score': 0.5462912917137146}


413it [00:19, 20.70it/s]


KeyboardInterrupt: 

In [27]:
ice

,text,score,skt/kogpt2-base-v2,beomi/kcbert-large
0,알차네요 무엇보다도 빨리들을 수 있어 좋습니다,1,0,0
1,명불허전 명강사라 그런지 말씀을 참 잘하시네요~,1,0,0
2,강추 30년 가까이 일하면서도 놓치고 있었던 것들을 알려주었습니다. 그리고 무조건 ...,1,0,0
3,패들렛과 띵커벨 활용 잘 할 수 있어요 패들렛과 띵커벨 활용하고 있었으나 다양한 교...,1,0,0
4,"필수 연수 부담 없이 편안하게 안전 연수 올 해 꼭 받아야 하는데, 부담 없이 편안...",1,0,0
...,...,...,...,...
5236,알베르토가 함께하는 핵심역량 연계 다문화 교육 여타 연수와 다르게 새로 알게된 내용...,1,0,0
5237,전문가가 알려주는 창의 영재교육의 비법 연수 후기 영재 교육에 대한 틀을 잡아주시는...,1,0,0
5238,도도한 교사생활 연수를 통해 자신감을 더 갖게 되었다.,1,0,0
5239,교실에서 활용할 수 있는 앱들 중심으로 재미있게 들었어요. 교실에서 활용할 수 있는...,1,0,0


In [14]:
ice['beomi/kcbert-large'].value_counts()

0    5241
Name: beomi/kcbert-large, dtype: int64

In [23]:
ice.drop(columns=['jaehyeong/koelectra-base-v3-generalized-sentiment-analysis', 
                  'facebook/bart-large-mnli', 'google/electra-small-discriminator'], inplace=True)

In [26]:
ice['beomi/kcbert-large'].value_counts()

0    5241
Name: beomi/kcbert-large, dtype: int64